In [ ]:
import tensorflow as tf
""" <<LSTM 모형을 살펴 보자 >> """
"""
 - 셀상태: LSTM의 핵심은 셀 상태라고 하는 내부 상태를 통해 정보를 장기간 저장할 수 있다는 점이다.
          이 셀 상태는 네트워크를 따라 길게 연결되어 있으며, 중요한 정보를 유지하고 불필요한 정보를 제거하는 게이트를 통해 조정된다.
 - 게이트(Gates): 정보를 셀 상태에 추가하거나 제거하기 위해 만들어진 구조. (시그모이드 + 포인트와이즈 곱셈연산)
     Forget Gate, Input Gate, Output Gate로 구성.
     1) 망각 게이트: 어떤 정보를 유지하거나 잊을지를 결정함. 
                   이전 숨겨진 상태와 현재 입력을 살펴보고, 시그모이드 함수를 통해 전달합니다. 0은 "완전히 잊음"을 의미하고 1은 "완전 유지"를 의미.
                   이전 상태와 현재 입력을 바탕으로 어떤 정보를 "잊어야" 할지를 결정하는 시그모이드 함수를 사용합니다.
                      Xt(현재 입력) + h(t-1)(숨겨진거) 를 받아서 시그모이드로 연산 후, 각 요소에 따라 정보량을 나타낸다.
     2) 입력 게이트: 셀 상태에 어떤 새로운 정보를 추가할지 결정합니다. 
                   여기에는 업데이트할 값을 결정하는 sigmoid 레이어와 상태에 추가할 수 있는 새로운 후보 값의 벡터를 생성하는 tanh 레이어가 포함됩니다.
                   이는 새로운 후보값들을 생성하는 tanh 레이어와 이 후보값들 중 어떤 것을 실제로 업데이트할지 결정하는 시그모이드 레이어로 구성됩니다. ★
                      현재 입력과 이전의 입력을 받아서, 시그모이드 연산 후, 새로운 정보의 어떤 부분을 업데이트할 건지 결정.
                      새로운 셀 상태는 같은 입력을 받지만 하이퍼볼릭탄젠트 함수로 통과시킨다.
                      입력게이트의 출력과 새로운 셀 상태는 요소별 곱셈을 통해서 결합한다.
     3) 출력 게이트: 셀 상태로 다음 hidden state를 결정한다. 즉,셀 상태의 어떤 부분이 출력되는지 결정합니다.
                   여기에는 tanh 함수(값을 -1과 1 사이가 되도록 푸시)를 통해 셀 상태를 필터링한 다음 이 값에 시그모이드 게이트의 출력을 곱하는 작업이 포함됩니다.
                     Xt + h_(t-1)를 받아서 시그모이드 연산과 동시에 하이퍼볼릭탄젠트 함수 결과로 -1~1 사이로 조정한 값이 리턴.
                     -> 출력 게이트의 출려과 탄젠트 함수를 통과한 셀 상태는 요소별 곱셈을 통해 결합되며 최종 ht를 생성해서 리턴한다.
 - 업데이트 상태: 출력게이트는 업데이트 된 셀 상태의 어떤 부분을 출력할 지 결정하고 최종 출력을 생성한다.
 
LSTM의 장점
 - 장기 의존성 학습: LSTM은 장기 의존성 문제를 해결할 수 있어, 긴 시퀀스 데이터에서도 이전의 중요한 정보를 기억할 수 있습니다.
 - 다양한 응용 분야: 자연어 처리(NLP), 음성 인식, 시계열 분석 등 다양한 분야에서 효과적으로 사용될 수 있습니다.
 - 유연성: 가변적인 길이의 시퀀스 데이터를 처리할 수 있으며, 다양한 형태의 입력 데이터에 적응할 수 있습니다.
 
LSTM의 단점 
LSTM은 복잡한 시퀀스 데이터를 다룰 수 있는 강력한 모델이지만, 계산 비용이 많이 들고 (computationally intensive) 훈련이 느리며,
매개변수를 주의 깊게 조정해야 하며, 특히 소규모 데이터 세트에서는 과적합이 발생할 수 있습니다.
때로는 더 새롭고 효율적인 모델(예: Transformers, Attention mechanisms)에 비해 성능이 떨어질 수 있습니다.
그럼에도 불구하고, LSTM은 여전히 많은 시퀀스 모델링 작업에서 중요한 역할을 합니다.
"""

LSTM 논문: https://www.bioinf.jku.at/publications/older/2604.pdf


1) Embedding : 복수의 문장을 입력했을 때 , 문장의 단어를 벡터 표현으로 변환 한 다음 리턴해준다. 

2) GlobalAveragePooling1D : Embedding에서 문장을 벡터로 리턴하게 되면 단어 벡터의 차원별로 평균을 리턴. 각 문장의 특징을 압축한 데이터로 표현할 수 있다.  
https://colah.github.io/posts/2015-08-Understanding-LSTMs/

In [ ]:
tf.keras.layers.Embedding(
    input_dim,
    output_dim,
    embeddings_initializer='uniform',
    embeddings_regularizer=None,
    activity_regularizer=None,
    embeddings_constraint=None,
    mask_zero=False,
    input_length=None,
    **kwargs
)

tf.keras.layers.GlobalAveragePooling1D(
    data_format='channels_last', **kwargs
)

<img src='etc/lstm_gates.PNG' width='66.66%' align='left'></img>

 - LSTM 셀은 이전 은닉상태 (ht-1)과 현재 단어 임베딩(xt)가 입력되면 새로운 은닉상태 ht를 출력한다. 
 - ht의 길이는 LSTM에 있는 유닛의 개수와 동일하다.  
 - LSTM 층은 하나의 셀이(Ct) 있고 여러개의 유닛을 포함한다.
 

In [ ]:
https://arxiv.org/pdf/1505.04597.pdf 유넷